# 🛡️ Complete Phishing Email Detection System

This notebook implements a complete machine learning pipeline for phishing email detection using:
- **K-Nearest Neighbors (KNN)**
- **Support Vector Machine (SVM)**

## 📋 Features
- Data loading and exploration
- Comprehensive data preprocessing
- Model training and optimization
- Performance evaluation and visualization
- Model persistence (saves trained models)

---

## 🔧 Step 1: Install Required Dependencies

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn joblib -q

## 📚 Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries imported successfully!")

## 📂 Step 3: Load the Dataset

**Note:** Make sure to upload the `emails.csv` file to your Colab environment first!

You can upload it using:
1. The file upload button in the left sidebar
2. Or run the cell below to use the file upload widget

In [ ]:
# Uncomment the lines below if you need to upload the file
# from google.colab import files
# uploaded = files.upload()

# For this notebook, we'll use the path where you've uploaded the file
# Update this path based on where you uploaded your emails.csv file
DATA_PATH = 'emails.csv'  # Change this to your file path if different

print(f"Loading data from: {DATA_PATH}")

## 📊 PHASE 1: Data Loading and Exploration

In [ ]:
print("="*60)
print("PHASE 1: LOADING AND EXPLORING DATA")
print("="*60)

# Load the dataset
df = pd.read_csv(DATA_PATH)

# Display basic information
print(f"\nDataset Shape: {df.shape}")
print(f"Number of emails: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

print("\nFirst few rows:")
display(df.head())

print("\nColumn names (first 10):")
print(df.columns.tolist()[:10])
print(f"... and {len(df.columns) - 10} more columns")

# Check for the target column
target_column = 'Prediction'  # Based on the emails.csv structure

if target_column in df.columns:
    print(f"\nTarget column found: '{target_column}'")
    print("\nClass Distribution:")
    print(df[target_column].value_counts())
    
    # Visualize class distribution
    plt.figure(figsize=(8, 5))
    df[target_column].value_counts().plot(kind='bar', color=['green', 'red'])
    plt.title('Email Classification Distribution')
    plt.xlabel('Email Type')
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
else:
    print("\nWarning: Could not identify target column automatically")

# Check for missing values
missing_total = df.isnull().sum().sum()
print(f"\nTotal Missing Values: {missing_total}")

if missing_total == 0:
    print("✓ No missing values found - dataset is clean!")
else:
    print(f"⚠ Found {missing_total} missing values")

# Get statistical summary
print("\nStatistical Summary (first 5 features):")
display(df.describe().iloc[:, :5])

## 🔄 PHASE 2: Data Preprocessing

In [ ]:
print("="*60)
print("PHASE 2: DATA PREPROCESSING")
print("="*60)

# Separate features (X) and target variable (y)
# Remove 'Email No.' if it exists, and the target column
columns_to_drop = [target_column]
if 'Email No.' in df.columns:
    columns_to_drop.append('Email No.')

X = df.drop(columns_to_drop, axis=1)
y = df[target_column]

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("\nOriginal labels:", y.unique())
print("Encoded labels:", np.unique(y_encoded))
print("Mapping:", dict(zip(label_encoder.classes_, 
                          label_encoder.transform(label_encoder.classes_))))

# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"Training set class distribution: {np.bincount(y_train)}")
print(f"Testing set class distribution: {np.bincount(y_test)}")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nOriginal feature range (first feature):")
print(f"Min: {X_train.iloc[:, 0].min()}, Max: {X_train.iloc[:, 0].max()}")
print("\nScaled feature range (first feature):")
print(f"Min: {X_train_scaled[:, 0].min():.2f}, Max: {X_train_scaled[:, 0].max():.2f}")
print("✓ Feature scaling completed!")

## 🎯 PHASE 3: Building KNN Model

In [ ]:
print("="*60)
print("PHASE 3: BUILDING KNN MODEL")
print("="*60)

# Test different K values
k_values = range(1, 31, 2)
train_scores = []
test_scores = []

print("\nTesting different K values...")
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    train_scores.append(knn.score(X_train_scaled, y_train))
    test_scores.append(knn.score(X_test_scaled, y_test))

# Visualize the results
plt.figure(figsize=(10, 6))
plt.plot(k_values, train_scores, label='Training Accuracy', marker='o')
plt.plot(k_values, test_scores, label='Testing Accuracy', marker='s')
plt.xlabel('K Value')
plt.ylabel('Accuracy')
plt.title('KNN Performance vs K Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Find the best K
best_k = k_values[np.argmax(test_scores)]
print(f"\nOptimal K value: {best_k}")
print(f"Best testing accuracy: {max(test_scores):.4f}")

# Train final KNN model
knn_final = KNeighborsClassifier(n_neighbors=best_k)
knn_final.fit(X_train_scaled, y_train)

# Make predictions
y_pred_knn = knn_final.predict(X_test_scaled)

print("\n✓ KNN Model trained successfully!")

## 🚀 PHASE 4: Building SVM Model

In [ ]:
print("="*60)
print("PHASE 4: BUILDING SVM MODEL")
print("="*60)

# Test different kernels
kernels = ['linear', 'rbf', 'poly']
svm_results = {}

print("\nTesting different SVM kernels...")
for kernel in kernels:
    print(f"\nTesting SVM with {kernel} kernel...")
    
    svm = SVC(kernel=kernel, random_state=42)
    svm.fit(X_train_scaled, y_train)
    
    y_pred = svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    svm_results[kernel] = accuracy
    
    print(f"{kernel.capitalize()} kernel accuracy: {accuracy:.4f}")

best_kernel = max(svm_results, key=svm_results.get)
print(f"\nBest performing kernel: {best_kernel}")
print(f"Best accuracy: {svm_results[best_kernel]:.4f}")

# Fine-tune the best model
print("\nPerforming grid search for optimal parameters...")

if best_kernel == 'rbf':
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.001, 0.01, 0.1, 1]
    }
elif best_kernel == 'linear':
    param_grid = {
        'C': [0.1, 1, 10, 100]
    }
else:  # poly
    param_grid = {
        'C': [0.1, 1, 10],
        'degree': [2, 3, 4]
    }

svm = SVC(kernel=best_kernel, random_state=42, probability=True)

grid_search = GridSearchCV(
    svm,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# Use the best model
svm_final = grid_search.best_estimator_
y_pred_svm = svm_final.predict(X_test_scaled)

print("\n✓ SVM Model trained successfully!")

## 📈 PHASE 5: Model Evaluation

In [ ]:
# KNN Evaluation
print("="*60)
print("K-NEAREST NEIGHBORS EVALUATION")
print("="*60)

knn_accuracy = accuracy_score(y_test, y_pred_knn)
print(f"\nAccuracy: {knn_accuracy:.4f} ({knn_accuracy*100:.2f}%)")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_knn,
                           target_names=['Legitimate', 'Phishing']))

knn_cm = confusion_matrix(y_test, y_pred_knn)
print("\nConfusion Matrix:")
print(knn_cm)
print(f"\nTrue Negatives (Correct Legitimate): {knn_cm[0][0]}")
print(f"False Positives (Legitimate marked as Phishing): {knn_cm[0][1]}")
print(f"False Negatives (Phishing marked as Legitimate): {knn_cm[1][0]}")
print(f"True Positives (Correct Phishing): {knn_cm[1][1]}")

In [ ]:
# SVM Evaluation
print("="*60)
print("SUPPORT VECTOR MACHINE EVALUATION")
print("="*60)

svm_accuracy = accuracy_score(y_test, y_pred_svm)
print(f"\nAccuracy: {svm_accuracy:.4f} ({svm_accuracy*100:.2f}%)")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm,
                           target_names=['Legitimate', 'Phishing']))

svm_cm = confusion_matrix(y_test, y_pred_svm)
print("\nConfusion Matrix:")
print(svm_cm)
print(f"\nTrue Negatives (Correct Legitimate): {svm_cm[0][0]}")
print(f"False Positives (Legitimate marked as Phishing): {svm_cm[0][1]}")
print(f"False Negatives (Phishing marked as Legitimate): {svm_cm[1][0]}")
print(f"True Positives (Correct Phishing): {svm_cm[1][1]}")

## 📊 PHASE 6: Visualizations

In [ ]:
# Confusion Matrices Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# KNN Confusion Matrix
sns.heatmap(knn_cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Legitimate', 'Phishing'],
            yticklabels=['Legitimate', 'Phishing'],
            ax=axes[0])
axes[0].set_title(f'KNN Confusion Matrix\nAccuracy: {knn_accuracy:.4f}')
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

# SVM Confusion Matrix
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Legitimate', 'Phishing'],
            yticklabels=['Legitimate', 'Phishing'],
            ax=axes[1])
axes[1].set_title(f'SVM Confusion Matrix\nAccuracy: {svm_accuracy:.4f}')
axes[1].set_ylabel('Actual')
axes[1].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

In [ ]:
# Model Comparison Bar Chart
plt.figure(figsize=(8, 6))
models = ['KNN', 'SVM']
accuracies = [knn_accuracy, svm_accuracy]
colors = ['skyblue', 'lightgreen']

bars = plt.bar(models, accuracies, color=colors, edgecolor='black', linewidth=1.5)
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.ylim([min(accuracies) - 0.05, 1.0])

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}',
            ha='center', va='bottom', fontweight='bold')

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Print comparison
print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)
print(f"\nKNN Accuracy: {knn_accuracy:.4f} ({knn_accuracy*100:.2f}%)")
print(f"SVM Accuracy: {svm_accuracy:.4f} ({svm_accuracy*100:.2f}%)")
print(f"\nBest Model: {'SVM' if svm_accuracy > knn_accuracy else 'KNN'}")
print(f"Performance Difference: {abs(svm_accuracy - knn_accuracy)*100:.2f}%")

## 🔍 PHASE 7: Feature Analysis

In [ ]:
print("="*60)
print("FEATURE ANALYSIS")
print("="*60)

# Calculate mean feature values for each class
legitimate_emails = X_train[y_train == 0]
phishing_emails = X_train[y_train == 1]

feature_diff = abs(legitimate_emails.mean() - phishing_emails.mean())
top_features = feature_diff.nlargest(20)

plt.figure(figsize=(10, 8))
top_features.plot(kind='barh')
plt.xlabel('Absolute Difference in Mean Values')
plt.title('Top 20 Most Discriminative Features')
plt.tight_layout()
plt.show()

print("\nTop 10 most discriminative features:")
for i, (feature, value) in enumerate(top_features.head(10).items(), 1):
    print(f"{i}. {feature}: {value:.4f}")

## 🧪 PHASE 8: Test Predictions

In [ ]:
print("="*60)
print("TESTING PREDICTIONS ON SAMPLE EMAILS")
print("="*60)

# Test with sample emails from test set
n_samples = 5
sample_indices = np.random.choice(len(X_test), n_samples, replace=False)

for i, idx in enumerate(sample_indices, 1):
    print(f"\n--- Sample Email {i} ---")
    
    sample_email = X_test.iloc[idx].values.reshape(1, -1)
    sample_scaled = scaler.transform(sample_email)
    
    # Make predictions
    knn_pred = knn_final.predict(sample_scaled)[0]
    svm_pred = svm_final.predict(sample_scaled)[0]
    
    # Get probabilities
    knn_proba = knn_final.predict_proba(sample_scaled)[0]
    svm_proba = svm_final.predict_proba(sample_scaled)[0]
    
    actual = y_test[idx]
    
    print(f"Actual: {'Phishing' if actual == 1 else 'Legitimate'}")
    print(f"KNN Prediction: {'Phishing' if knn_pred == 1 else 'Legitimate'} (Confidence: {knn_proba[knn_pred]*100:.2f}%)")
    print(f"SVM Prediction: {'Phishing' if svm_pred == 1 else 'Legitimate'} (Confidence: {svm_proba[svm_pred]*100:.2f}%)")
    
    if knn_pred == actual and svm_pred == actual:
        print("✓ Both models predicted correctly!")
    elif knn_pred == actual or svm_pred == actual:
        print("⚠ One model predicted correctly")
    else:
        print("✗ Both models predicted incorrectly")

## 💾 PHASE 9: Save Trained Models

**This is the final step - saving all trained models and the scaler for future use.**

In [ ]:
print("="*60)
print("SAVING TRAINED MODELS")
print("="*60)

# Save the models
joblib.dump(knn_final, 'knn_phishing_detector.pkl')
joblib.dump(svm_final, 'svm_phishing_detector.pkl')
joblib.dump(scaler, 'feature_scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

print("\n✓ Models saved successfully!")
print("  - knn_phishing_detector.pkl")
print("  - svm_phishing_detector.pkl")
print("  - feature_scaler.pkl")
print("  - label_encoder.pkl")

# Verify saved models
import os
print("\n--- Verifying Saved Files ---")
for filename in ['knn_phishing_detector.pkl', 'svm_phishing_detector.pkl', 
                 'feature_scaler.pkl', 'label_encoder.pkl']:
    if os.path.exists(filename):
        size = os.path.getsize(filename) / 1024  # Size in KB
        print(f"✓ {filename} ({size:.2f} KB)")
    else:
        print(f"✗ {filename} - NOT FOUND")

## 📥 Download Models (Optional)

Run this cell to download the trained models to your local machine.

In [ ]:
# Uncomment to download models in Google Colab
# from google.colab import files
# files.download('knn_phishing_detector.pkl')
# files.download('svm_phishing_detector.pkl')
# files.download('feature_scaler.pkl')
# files.download('label_encoder.pkl')
# print("✓ All models downloaded!")

## 📝 How to Use the Trained Models

To use the trained models for predictions on new emails:

In [ ]:
print("="*60)
print("USAGE EXAMPLE")
print("="*60)

print("""
To use these models for future predictions:

```python
import joblib
import numpy as np

# Load the models
knn_model = joblib.load('knn_phishing_detector.pkl')
svm_model = joblib.load('svm_phishing_detector.pkl')
scaler = joblib.load('feature_scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Prepare your email features (must match the training features)
# your_email_features should be a list or array with the same number of features
your_email_features = [...]  # Your feature vector

# Scale the features
email_scaled = scaler.transform([your_email_features])

# Make predictions
knn_prediction = knn_model.predict(email_scaled)[0]
svm_prediction = svm_model.predict(email_scaled)[0]

# Get probabilities
knn_proba = knn_model.predict_proba(email_scaled)[0]
svm_proba = svm_model.predict_proba(email_scaled)[0]

# Display results
print(f"KNN: {'Phishing' if knn_prediction == 1 else 'Legitimate'}")
print(f"SVM: {'Phishing' if svm_prediction == 1 else 'Legitimate'}")
```
""")

print("\n" + "="*60)
print("PROJECT COMPLETED SUCCESSFULLY!")
print("="*60)
print(f"\n📊 Final Results:")
print(f"   KNN Accuracy: {knn_accuracy*100:.2f}%")
print(f"   SVM Accuracy: {svm_accuracy*100:.2f}%")
print(f"   Best Model: {'SVM' if svm_accuracy > knn_accuracy else 'KNN'}")
print(f"\n💾 All models have been trained and saved!")
print(f"   - Total emails processed: {len(df)}")
print(f"   - Features used: {X.shape[1]}")
print(f"   - Training samples: {len(X_train)}")
print(f"   - Testing samples: {len(X_test)}")